# Doc2Vec Searching of Lang Database

Use `gensim` and a simple `doc2vec` model trained on stories from the Lang coloured fairy books to support semantic retrieval of fairy stories.

The approach can be summarised as follows:

- generate a vocabulary of terms representative of the search corpus;
- generate a vector space where each dimension is a word in the vocabulary;
- generate a vector for each document or search phrase;
- retrieve documents based on similarity between document vector and search phrase vector.

The following recipe is inspired by [How to make a search engine on Movies Description](https://github.com/ppontisso/Text-Search-Engine-using-Doc2Vec-and-TF-IDF/blob/master/notebook.ipynb).

## Connecting to the Database

We're going to work with our Lang fairy story database, so let's set up a connection to it:

In [1]:
from sqlite_utils import Database

db_name = "demo.db"

db = Database(db_name)

Let's remind ourselves of the database structure:

In [2]:
print(db.schema)

CREATE TABLE [books] (
   [book] TEXT,
   [title] TEXT,
   [text] TEXT,
   [last_para] TEXT,
   [first_line] TEXT,
   [provenance] TEXT,
   [chapter_order] INTEGER,
   PRIMARY KEY ([book], [title])
);
CREATE TABLE [books_metadata] (
   [title] TEXT,
   [year] INTEGER
);
CREATE VIRTUAL TABLE [books_fts] USING FTS5 (
    [title], [text],
    content=[books]
);
CREATE TABLE 'books_fts_data'(id INTEGER PRIMARY KEY, block BLOB);
CREATE TABLE 'books_fts_idx'(segid, term, pgno, PRIMARY KEY(segid, term)) WITHOUT ROWID;
CREATE TABLE 'books_fts_docsize'(id INTEGER PRIMARY KEY, sz BLOB);
CREATE TABLE 'books_fts_config'(k PRIMARY KEY, v) WITHOUT ROWID;
CREATE TRIGGER [books_ai] AFTER INSERT ON [books] BEGIN
  INSERT INTO [books_fts] (rowid, [title], [text]) VALUES (new.rowid, new.[title], new.[text]);
END;
CREATE TRIGGER [books_ad] AFTER DELETE ON [books] BEGIN
  INSERT INTO [books_fts] ([books_fts], rowid, [title], [text]) VALUES('delete', old.rowid, old.[title], old.[text]);
END;
CREATE TRIGGER 

Recall that we can perform a full text search:

In [3]:
#q = 'king "three sons" gold'
q = 'hansel witch'
_q = f'SELECT title FROM books_fts WHERE books_fts MATCH {db.quote(q)} ;'

for row in db.query(_q):
    print(row["title"])

Hansel And Grettel


We can randomly sample a selection of rows with a query of the following form:

In [4]:
# Via https://gist.github.com/alecco/9976dab8fda8256ed403054ed0a65d7b

_q_random_sample = """
SELECT * FROM books
WHERE rowid IN (SELECT rowid FROM books
                WHERE title NOT LIKE "Preface"
                ORDER BY random() LIMIT {});
"""

for row in db.query(_q_random_sample.format(5)):
    print(row["title"])

Grasp All, Lose All
The Sea King's Gift
The Three Brothers
The Cottager And His Cat
The Golden Mermaid


## Simple Model

We could use an off-the-shelf model to process documents, or we can train our own model from our own documents so that the word vectors are aligned to our dataset. In a large corpus, we can train on a sample of documents if they are representative of the whole.

If we train against the whole dataset, we can search into the dataset directly from the model. If train the model on a partial collection, then we can only compare search phrases and documents that we have generated vectors for.

To create the model, it helps if we clean the documents, e.g. by decasing, and removing punctuation:

In [5]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_numeric, remove_stopwords

def clean_text(text):
    """Generate a cleaned, tokenised version of a text."""
    CUSTOM_FILTERS = [lambda x: x.lower(),
                      strip_tags, strip_punctuation,
                      strip_numeric, remove_stopwords]
    
    return preprocess_string(text, CUSTOM_FILTERS)

Apply the cleaning function to the text on the way in to creating the training corpus:

In [6]:
sample_corpus = db.query(_q_random_sample.format(9999))

sample_docs = [(clean_text(r['text']),
               f"{r['book']}::{r['title']}", #create a unique tag
               r['title'])
               for r in sample_corpus]

# For the first doc, preview the first 5 cleaned words and title
sample_docs[0][0][:5], sample_docs[0][1], sample_docs[0][2]

(['time', 'certain', 'country', 'lived', 'king'],
 'The Blue Fairy Book::The Bronze Ring',
 'The Bronze Ring')

In [7]:
# The gensim model needs named tuples
# including at least a words and tags dimension
# Naively we can just use a document index count as the tag
from collections import namedtuple

StoryDoc = namedtuple('StoryDoc',
                      'words tags title')

sample_docs_training = []

for i, sample_doc in enumerate(sample_docs):
    sample_docs_training.append(StoryDoc(sample_doc[0],
                                         [sample_doc[1]],
                                         sample_doc[2]))

In [8]:
from gensim.models import Doc2Vec

# Define the parameters for building the model.
# We can also pass a list of documents
# via the first "documents" parameter
# and the model will be trained against those.
# Alternatively, create an empty model and train it later.
model = Doc2Vec(
                # dm: training algorithm;
                # 1: distributed memory/PV-DM;
                # 0: distributed bag of words (PV-DBOW)
                dm=1,
                # vector_size: size of feature vectors
                vector_size=300,
                # window: max dist between current & predicted word
                window=10,
                # hs: 1: hierarchical softmax;
                # hs: 0 : negative sampling if negative
                hs=0,
                # min_count: ignore words w/ lower frequency
                # There is a risk to setting this too high
                # particularly if a search term is likely unique,
                # as it might be with a name. On the other hand,
                # for such situations, a simple search might be better?
                min_count=1,
                # sample: randomly downsample hi-frequnecy words
                # useful range: (0, 1e-5)
                sample=1e-5,
                )

The model is built around a vocabulary extracted from the training document corpus.

In [9]:
# Build the model vocabulary
model.build_vocab(sample_docs_training)

We can now train the model (this may take some time for a large corpus):

In [10]:
# It would be useful if we could display a progress bar for this...
model.train(sample_docs_training,
            total_examples=model.corpus_count,
            epochs=100, start_alpha=0.01, end_alpha=0.01)

Rather than creating a model each time we want to use it, we can save the model and then load it as required:

In [11]:
# Save a model
model.save("lang_model.gensim")

# Load in a model
model = Doc2Vec.load("lang_model.gensim")

To retrieve a document matching a search phrase, we need to encode the search phrase and then try to find a matching document:

In [12]:
search_phrase = """
hansel and his sister were cast out by their wicked stepmother
and went into forest and met an evil witch
"""

# Preprocess the search phrase
tokens = clean_text(search_phrase)

tokens

['hansel',
 'sister',
 'cast',
 'wicked',
 'stepmother',
 'went',
 'forest',
 'met',
 'evil',
 'witch']

Generate a vector for the tokens:

In [13]:
# Generate the vector representation from the model
search_vector = model.infer_vector(tokens, alpha=0.001, steps = 5)

We can now search for related documents from the original training set based on how well their vectors match the vector generated for the search phrase:

In [14]:
# Find the top 5 matches
matches = model.docvecs.most_similar([search_vector], topn=10)
# To rank every document from the training corpus
# set: topn=model.docvecs.count

# The response gives the original training document ids and match scores
matches

[('The Yellow Fairy Book::How To Tell A True Princess', 0.530689537525177),
 ('The Yellow Fairy Book::In The Land Of Souls', 0.4511426091194153),
 ('The Lilac Fairy Book::The Raspberry Worm', 0.4241422712802887),
 ('The Lilac Fairy Book::Little Lasse', 0.41359323263168335),
 ('The Blue Fairy Book::Rumpelstiltzkin', 0.40738600492477417),
 ('The Olive Fairy Book::Kupti And Imani', 0.4026232659816742),
 ('The Yellow Fairy Book::A Story About A Darning-Needle',
  0.40114545822143555),
 ('The Pink Fairy Book::The Shirt-Collar', 0.3960583806037903),
 ('The Lilac Fairy Book::A Fish Story', 0.38714125752449036),
 ('The Blue Fairy Book::Hansel And Grettel', 0.38626253604888916)]

Let's try another one:

In [15]:
search_phrase = """
a poor orphan girl lives with her wicked stepmother and sisters
but then her fairy godmother appears and she goes to a ball 
and leaves at midnight
but loses her slipper then finally marries the prince
"""

# Preprocess the search phrase
tokens = clean_text(search_phrase)
search_vector = model.infer_vector(tokens, alpha=0.01, steps = 50)
model.docvecs.most_similar([search_vector], topn=10)

[('The Blue Fairy Book::Cinderella, Or The Little Glass Slipper',
  0.5241366624832153),
 ('The Blue Fairy Book::Rumpelstiltzkin', 0.5099612474441528),
 ('The Blue Fairy Book::Little Red Riding Hood', 0.4753239154815674),
 ('The Brown Fairy Book::Habogi', 0.47470545768737793),
 ('The Pink Fairy Book::Peter Bull', 0.4617704749107361),
 ('The Pink Fairy Book::The Merry Wives', 0.45907604694366455),
 ('The Crimson Fairy Book::Lovely Ilonka', 0.4566786587238312),
 ('The Yellow Fairy Book::The Swineherd', 0.4557437598705292),
 ('The Red Fairy Book::The Six Sillies', 0.4503322243690491),
 ('The Green Fairy Book::Spindle, Shuttle, And Needle', 0.4496467113494873)]

Not that the result is stochastic (has a random element) in the way that the search vector is inferred: if you rerun the query, you will likely generate a different search vector. As a consequence, the search results returned are likely differ in their order and match scores each time the query is run.

## Creating a Search Tool

The next step is to register a custom SQLite function that will generate a vector for a search term and return matching records on that basis.

In [16]:
from vtfunc import TableFunction

class SQLite_doc2Vec_Search(TableFunction):
    params = ['query', 'num', 'threshold']
    columns = ['book', 'title', 'score']
    name = 'doc2vec_model_search'
    
    model = Doc2Vec.load("lang_model.gensim")
    
    def initialize(self, query=None, num=None, threshold=None):
        num = 10 if num is None else num

        tokens = clean_text(query)
        search_vector = model.infer_vector(tokens, alpha=0.01, steps = 50)
        scores = model.docvecs.most_similar([search_vector],
                                            topn=model.docvecs.count)
        if threshold:
            scores = [(t, s) for (t, s) in scores if s >= threshold ]

        self._iter = iter(scores)
 
    def iterate(self, idx):
        (tag, score) = next(self._iter)
        items = tag.split("::")
        return (items[0], items[1], score,)

# And register the function
SQLite_doc2Vec_Search.register(db.conn)

The query searches over the model and can take various forms:

- `doc2vec_model_search("search phrase")`
- `doc2vec_model_search("search phrase", NUM_OF_RESULTS)`
- `doc2vec_model_search("search phrase", NUM_OF_RESULTS, MIN_SCORE)`
- `doc2vec_model_search("search phrase", NULL, MIN_SCORE)`

For example:

In [17]:
model_query = f"""
SELECT *
FROM doc2vec_model_search('''{search_phrase}''', NULL, 0.45 );
"""

for i in db.execute(model_query):
    print(i)
 

('The Blue Fairy Book', 'Cinderella, Or The Little Glass Slipper', 0.505182147026062)
('The Blue Fairy Book', 'Rumpelstiltzkin', 0.48724472522735596)
('The Crimson Fairy Book', 'Lovely Ilonka', 0.47657108306884766)
('The Green Fairy Book', 'Spindle, Shuttle, And Needle', 0.46215206384658813)
('The Yellow Fairy Book', 'How To Tell A True Princess', 0.4570574462413788)
('The Red Fairy Book', 'The Six Sillies', 0.45611968636512756)
('The Brown Fairy Book', 'Habogi', 0.4532594382762909)


## Saving Model Vectors into the Database

If we look at the object type of one of the model vectors, we see that it is a `numpy.ndarray`, which can be easily represented as a list:

In [18]:
type(model.docvecs[0])

numpy.ndarray

We can store this data in the SQLite database as a `BLOB`. To simplify the process of converting the array into and out of the appropriate format for storage in the database compared to its use as a gensim vector, we can register a custom handler for the `numpy.ndarray` object:

In [19]:
# Via https://stackoverflow.com/a/18622264/454773
import sqlite3
import numpy as np
import io

def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)


# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

Now we need to reset the database to a connection that supports the custom handler we have just registered:

In [20]:
# Reset the database connection to used the parsed datatype
db.conn = sqlite3.connect(db_name, detect_types=sqlite3.PARSE_DECLTYPES)

We can now create a table with a custom "array" datatype:

In [21]:
# Give ourselves a clean slate
db["story_vectors"].drop(ignore=True)


# sqlite_utils doesn't appear to support custom types (yet?!)
# The following errors on the "array" datatype
"""
db["story_vectors"].create({
    "book": str,
    "title": str,
    "tag": str, # a unique key derived from book and title
    "vector": "array",
}, pk=("book", "title"),
    # The following is not currently supported by sqlite_utils
   #foreign_keys=[ (("book", "title"), "books", ("book", "title"))] # local-table-id, foreign-table, foreign-table-id]
)
"""

# so we can create a table the old fashioned way...
vector_table_create = """
CREATE TABLE story_vectors 
    (tag TEXT PRIMARY KEY, vector array, book TEXT, title TEXT );
"""

cur = db.conn.cursor()
cur.execute(vector_table_create)

We can generate a list of dictionaries, one per record used to train the model, that can then be added directly to the `story_vectors` database table:

In [22]:
xrecords = []

for tag in model.docvecs.doctags:
    xrecords.append({'book': tag.split('::')[0],
                     'title': tag.split('::')[1],
                     'tag': tag, 'vector':model.docvecs[tag]})

And add the records directly to the database:

In [23]:
db["story_vectors"].insert_all(xrecords)

<Table story_vectors (tag, vector, book, title)>

Let's pull an example record back showing just the first few elements of the vector associated with the record:

In [24]:
_q = f'SELECT * FROM story_vectors LIMIT 1;'

for row in db.query(_q):
    print(row['tag'], row['book'], row['title'], row['vector'][:10])

The Blue Fairy Book::The Bronze Ring The Blue Fairy Book The Bronze Ring [ 2.321162    0.7221756  -0.6747307  -2.32237    -1.3788195   0.29455236
 -0.57455003  0.3207829   1.6113997  -2.4436612 ]
